# Lab 6 - PySpark DataFrames.

# 0. Uruchomienie silnika Spark.

In [ ]:
# dla instalacji lokalnej

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_HOME'] = sys.executable

In [ ]:
# dla instalacji z dockerem

import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [1]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()
spark.sparkContext

<SparkContext master=local[2] appName=Create-DataFrame>

In [2]:
sc = spark.sparkContext

# 1. Spark DataFrame.

> Spark Dataframe API: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html

Zanim przejdziemy do obiektów typu DataFrame warto powiedzieć, że w systemie Spark występuje również tym Dataset, co może prowadzić do używania tych dwóch terminów zamiennie, co byłoby błędem. Obiekty typu Dataset są odrębnym typem i póki co nie są one dostępne w API Pythona dla Sparka, ale można na nich pracować z poziomu API Javy oraz języka Scala.
Kilka szczegółów na temat tego typu oraz jego tworzenia z poziomu języka Java lub Scala można znaleźć [tu](https://spark.apache.org/docs/3.5.3/sql-getting-started.html#creating-datasets) oraz [tu](https://spark.apache.org/docs/3.5.3/api/java/index.html).
Obiekty typu Dataset w języku Java i Scala są obiektami silnie typowanymi, więc mamy do dyspozycji transfomacje typowane, a w Pythonie są to transformacje nietypowane (z racji natury języka Python).

Spark DataFrame to rozproszona kolekcja danych Spark do pracy z danymi ustrukturyzowanymi, która podobna jest do obiektów DataFrame znanych z biblioteki pandas oraz języka R jednak dużo bardziej zoptymalizowana w kontekście pracy w środowisku rozproszonym. 


## Pobranie danych i wczytanie do ramki Spark.

In [ ]:
# pobranie spakowanego zbioru za pomocą polecenia systemowego wget
# strona datasetu: https://archive.ics.uci.edu/dataset/911/recipe+reviews+and+user+feedback+dataset
# to zadziała tylko w systemach Linux, w Windows można użyć polecenia curl
# !wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

# Windows
!curl -O https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# listujemy zawartość bieżącego folderu (również Linux, ale jak mamy WSL to i Windows)
!ls

# Windows
!dir

02_pyspark_dataframes.ipynb
readme.md


In [ ]:
# zmiana nazwy pliku - nie jest konieczna, ale trzeba zmienić później ścieżkę w kolejnej komórce notatnika
!mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

# Windows
! ren recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

In [7]:
# wypakowujemy plik do podfolderu data
import zipfile
with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")

In [ ]:
!ls ./data

# lub
# !dir ./data

Recipe Reviews and User Feedback Dataset.csv


In [ ]:
# sprawdzamy jak wyglądają 3 pierwsze linie pliku, widać, że pierwsza zawiera nagłówki kolumn a dane są oddzielone przecinkiem
# Linux, mac i WSL
!head -3 "data/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. Iâ€™ve written to them and asked them to please!, bring them back"


In [10]:
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")

## Wyświetlenie danych oraz schematu

In [11]:
# najpopularniejsza metoda ich pobrania to show(), ale jest ich więcej
df_reviews.show(5)

+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

In [12]:
# rzut oka na schemę tego DataFrame
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



In [13]:
# widać, że wszystkie kolumny są typu string, to jest domyślny sposób wczytywania danych przez spark z plain text
# możemy jednak przekazać dodatkowy parametr, który na podstawie próbki danych spróbuje dobrać typ danych odpowiedni dla kolumny
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",", inferSchema=True)

In [14]:
# po wypisaniu schemy widać zmianę
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- thumbs_up: integer (nullable = true)
 |-- thumbs_down: integer (nullable = true)
 |-- stars: integer (nullable = true)
 |-- best_score: integer (nullable = true)
 |-- text: string (nullable = true)



> Listę dostępnych typów danych znajdziesz tu: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html

In [15]:
# ramkę możemy również inicjalizować wskazując pożądane typy danych
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType

data = [("James","","Smith",36636,"M",3000),
    ("Michael","Rose","",40288,"M",4000),
    ("Robert","","Williams",42114,"M",4000),
    ("Maria","Anne","Jones",39192,"F",4000),
    ("Jen","Mary","Brown","","F",1000)
  ]

schema = StructType([ \
    StructField("firstname", StringType(), True), \
    StructField("user_id", StringType(), True), \
    StructField("lastname", StringType(), True), \
    StructField("id", StringType(), True), \
    # błąd konwersji "" na int!
    # StructField("id", LongType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True)
    # chcielibyśmy tak, ale tutaj nie da się za bardzo - błąd konwersji int na decimal!
    # StructField("salary", DecimalType(10,2), True) \
  ])

df_test = spark.createDataFrame(data=data,schema=schema)
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [16]:
# możemy wykonać rzutowanie po wczytaniu danych z większością kolumn typu tekstowego
import pyspark.sql.functions as F

df_test = df_test.withColumn("salary", F.col("salary").cast("decimal(10,2)"))
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [17]:
# wyświetlenie danych z pojedynczej kolumny
df_test.select(df_test.salary).show()

Py4JJavaError: An error occurred while calling o70.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 5) (LAPTOP-L9AQONII executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [18]:
# ile wierszy w ramce?
df_reviews.count()

18268

In [19]:
# DataFrame składa się z obiektów typu Column dla każdej kolumny
# API dla typu Column: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html

# do kolumn możemy się odwoływać tak jak w pandas API, ale wynik jest inny
df_reviews.user_name, df_reviews['user_name']

(Column<'user_name'>, Column<'user_name'>)

In [20]:
# aby wyświetlić dane musimy wywoałać funkcję select na obiekcie dataframe

df_reviews.select(df_reviews.user_name).show(5)

+----------+
| user_name|
+----------+
|   Jeri326|
|   Mark467|
|Barbara566|
|jeansch123|
|  camper77|
+----------+
only showing top 5 rows



In [ ]:
# do funkcji select możemy przekazać wiele kolumn a wywołania podobnie jak dla RDD są leniwe
print(df_reviews.select(df_reviews.user_name, df_reviews.user_reputation))
# musimy więc wywołać funkcję, której wykonanie "zmusi" Sparka do wyliczenia jej wartości lub jawnie wywołać np. show
df_reviews.select(df_reviews.user_name, df_reviews.user_reputation).show(5)

In [ ]:
# można zmienić to domyślne zachowanie Spark, ale zazwyczaj nie jest to dobry pomysł, chyba, że zbiór jest mały
# zmieniamy to poprzez edycję poniższego parametru
# spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
# lub indeksując kolumny innym sposobem
df_reviews.select(df_reviews['user_name'],df_reviews['user_reputation']).show(5)

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

# policzymy teraz liczbę wartości NULL w każdej kolumnie
df_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews.columns]).show()

## Filtrowanie danych

In [ ]:
# rzućmy okiem na kilka wierszy gdzie w kolumnie recipe_name jest wartość NULL
df_reviews.filter(df_reviews.recipe_code.isNull()).show()

In [ ]:
# zapisanie do nowej ramki danych bez wartości pustych
df_reviews_clean = df_reviews.na.drop()
df_reviews_clean.count()

In [ ]:
# dla pewności możemy to sprawdzić raz jeszcze
df_reviews_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews_clean.columns]).show()

In [ ]:
# filtrowanie danych z ramki
df_reviews_clean.filter(df_reviews.user_name.startswith('a')).select(df_reviews_clean.user_name).show(10)
df_reviews_clean.filter(df_reviews.stars == 5).show(10)

In [ ]:
from pyspark.sql.functions import avg

# wyliczenie średniej wartości z kolumny
df_reviews_clean.select(avg(df_reviews_clean.thumbs_up)).show()

In [ ]:
# ale możemy się dowiedzieć tego i więcej w sposób podobny do tego z biblioteki pandas
df_reviews_clean.select(df_reviews_clean.thumbs_up).describe().show()

In [ ]:
from pyspark.sql.functions import desc

df_reviews_clean.groupby('recipe_code').agg({'thumbs_down': 'sum'}).sort(desc('sum(thumbs_down)')).show(10)

Dla potrzeb laboratorium została stworzona funkcja, która pozwoli na generowanie datasetu i zapisane go w pliku csv na początek, aby zaprezentować podstawowe typy danych Sparka.

In [ ]:
# deklaracja zbiorów wartości dla poszczególnych kolumn przyszłego zbioru danych
header = ['id', 'firstname', 'lastname', 'age', 'salary']
firstnames = ['Adam', 'Katarzyna', 'Krzysztof', 'Marek', 'Aleksandra', 'Zbigniew', 'Wojciech', 'Mieczysław', 'Agata', 'Wisława']
lastnames = ['Mieczykowski', 'Kowalski', 'Malinowski' , 'Szczaw', 'Glut', 'Barański', 'Brzęczyszczykiewicz', 'Wróblewski', 'Wlotka', 'Pysla']
age = {'min': 18, 'max': 68}
salary = {'min': 3200, 'max': 12500}

In [ ]:
# funkcja do generowania fikcyjnego datasetu
# n_rows oznacza ilość wierszy, którą chcemy finalnie uzyskać


import random
from tqdm import tqdm

def build_dataset(filename, n_rows=100, chunk_size=100000):
    rows = []
    rows.append(header)
    mu = (salary['max'] + salary['min']) / 2
    sigma = 1000

    with open(filename, 'w', encoding='utf-8') as filehandler:
        
        for id in tqdm(range(1, n_rows + 1), total=n_rows, desc="Building dataset..."):
            row = [
                f'{id}', 
                f'{random.choice(firstnames)}', 
                f'{random.choice(lastnames)}', 
                f"{random.randint(age['min'], age['max'])}",
                f"{round(float(random.normalvariate(mu=mu, sigma=sigma)), 2)}"
            ]
            rows.append(row)
            if id % chunk_size == 0:
                filehandler.writelines([f"{','.join(row)}\n" for row in rows])
                rows = []


In [ ]:
# około 715MB zostanie zapisanych w pliku csv, dostosuj ilość rekordów do swoich potrzeb
build_dataset('employee.csv', 20_000_000)

In [ ]:
%%time
# więcej magicznych metod w Jupyter Notebooku: https://ipython.readthedocs.io/en/stable/interactive/magics.html
# wczytanie pliku csv przez spark
# df = spark.read.csv('employee.csv', header=True)
df = spark.read.csv('employee.csv', header=True, inferSchema=True)

In [ ]:
type(df)

In [ ]:
%%time
# wypisujemy schemat i 10 pierwszych wierszy utworzonego obiektu Spark DataFrame
df.printSchema()
df.show(10)

In [ ]:
# przykład wykorzystania funkcji transform, która mapuje wykonanie stworzonej funkcji tu_upper_str_columns na istniejącą kolumnę
# i zwraca nową ramkę z dodatkową kolumną
from pyspark.sql.functions import upper

def to_upper_str_columns(df, column_name, new_column_name):
    return df.withColumn(new_column_name, upper(df[column_name]))

df = df.transform(to_upper_str_columns, "firstname", "firstname_upper")

In [ ]:
df.show(10)

In [ ]:
# filtrowanie numeryczne, ale tu na kolumnie typu str - czy jest poprawne?
df.filter(df["salary"] > 10000).count()

In [ ]:
# na ile partycji została nasza ramka danych rozrzucona po "klastrze"?
df.rdd.getNumPartitions()

In [ ]:
%%time
# mierzymy czas operacji przy domyślnej liczbie partycji
df.filter(df["salary"] > 10000).count()

In [ ]:
df = df.repartition(12)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
%%time
# mierzymy czas operacji przy 12 partycjach dla 20_000_000 rekordów
df.filter(df["salary"] > 10000).count()

In [ ]:
# macierz częstości dla dwóch kolumn - uwaga dla bardzo różnorodnych danych!
df.crosstab("firstname", "age").sort("firstname_age").show()

In [ ]:
# funkcja explain może przydać się w przypadku bardziej zaawansowanego debuggingu, optymalizacji i zrozumienia
# kolejności działania niektórych elementów silnika Spark
query = df.filter(df.firstname.contains('ski'))
query.explain(mode='formatted')

In [ ]:
# zapisujemy ramkę do plików parquet
# zwróć uwagę na liczbę utworzonych plików

df.write.parquet('./data/parquet/')

In [ ]:
# lub chcąc nadpisać już istniejące dane - w trybie overwrite
df.write.mode("overwrite").parquet('./data/parquet/')

In [ ]:
sc.stop()

### Zadania

**Zadanie 1**  
Na zbiorze danych '_Recipe Reviews ..._' wykonaj:  
1.1  Zmień nazwę pierwszej kolumny z `_c0` na `id`.  
1.2  Wyświetl 10 najwyższych wartości w kolumnie `reply_count`.  
1.3  Wyświetl 10 najwyższych sum wartości w kolumnie `best_score` dla każdego przepisu (grupowanie).  
1.4  Które 10 przepisów miało najwięcej komentarzy?  
1.5  Wyświetl rozkład wartości w kolumnie `stars`.  


**Zadanie 2**  
Wczytaj zbiór danych `employee` nakazując Sparkowi wywnioskowanie bardziej optymalnych typów danych niż domyślny typ `string`.

**Zadanie 3**  
Jaki jest czas wykonania operacji `df.filter(df["salary"] > 10000).count()` tym razem przy numerycznym typie kolumny `salary`? Jest jakaś różnica?

**Zadanie 4**  
Wykorzystując przykład z dokumentacji klasy `Bucketizer` (https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Bucketizer.html) podziel dane w kolumnie `age` zbioru `employee` na buckety co 10 lat (10-19, 20-29, ..., 60-69) i wyświetl te dane dla 20 pierwszych wierzy w formie surowej oraz całość grupując po bucketach i licząc ile osób znalazło się w każdym z nich.